# Notebook to test stuff and create one-time things

## Map and graph related stuff

In [ ]:
# get xm and ym from clicking on map

import numpy as np, cv2 as cv, os, networkx
from os.path import join
from stuff import *

this_file = os.path.abspath('.')
# map_name, k = '2024_Big', K_BIG
# map_name, k = '2024_Medium', K_MEDIUM
map_name, k = '2024_Small', K_SMALL
# map_name, k = '2024_VerySmall', K_VERYSMALL
map_path = join(this_file, f'Simulator/src/models_pkg/track/materials/textures/{map_name}.png')
graph_path = join(this_file, 'stuff/track_graph.graphml')
g = networkx.read_graphml(graph_path)
map_img = cv.flip(cv.imread(map_path), 0)

def get_xy(event, x, y, flags, param):
    if event == cv.EVENT_LBUTTONDBLCLK:
        x, y = x, map_img.shape[0]-y # flip y to get the right coordinates
        xm, ym = pix2m(x, k), pix2m(y, k)
        print(f'xm: {xm:.3f}, ym: {ym:.3f}')

        #draw 2 circles on the map
        cv.circle(map_img, xy2cv(xm,ym,k), 5, (255, 0, 255), -1)
        cv.circle(map_img, xy2cv(xm,ym,k), 50, (255, 0, 255), 5)
    

cv.namedWindow('map', cv.WINDOW_NORMAL)
cv.resizeWindow('map', 1920//2-1, 600)

cv.setMouseCallback('map', get_xy)

while True:
    cv.imshow('map', cv.flip(map_img, 0))
    if cv.waitKey(10) == 27:
        break

cv.destroyAllWindows()

# roundabout center ~ xm: 16.049, ym: 10.530

In [ ]:
# create a perfect 16 around the roundabout
import numpy as np, matplotlib.pyplot as plt
xc, yc = 16.049, 10.530
r = 0.645
θs = np.linspace(0, 2*np.pi, 17)[:16]
xs, ys = xc+r*np.cos(θs), yc+r*np.sin(θs)
names = range(600, 616)
plt.plot(xs, ys, 'o')
plt.axis('equal')
plt.ylim(9.85, 11.3)
for x, y, name in zip(xs, ys, names):
    plt.text(x, y, name, fontsize=10)
plt.show()
for n, x, y in zip(names, xs, ys):
    print(f'{n} -> x: {x:.3f}, y: {y:.3f}')


In [ ]:
# show graph map
import cv2 as cv, os
from os.path import join
from stuff import *

this_file = os.path.abspath('.')
map_name, k = '2024_Small', K_SMALL

g = load_graph()

map_img = cv.imread(join(this_file, f'Simulator/src/models_pkg/track/materials/textures/{map_name}.png'))

np.random.seed(42)
for edge in g.edges:
    x1, y1 = g.nodes[edge[0]]['x'], g.nodes[edge[0]]['y']
    x2, y2 = g.nodes[edge[1]]['x'], g.nodes[edge[1]]['y']
    x1, y1 = x1, MAP_H_M-y1
    x2, y2 = x2, MAP_H_M-y2
    #generate random color with hsv, randomize only hue
    #set seed
    random_color = np.array([np.random.randint(0, 180), 150, 255], dtype=np.uint8).reshape(1,1,3)
    random_color = tuple(cv.cvtColor(random_color, cv.COLOR_HSV2BGR)[0,0].tolist()) # convert to bgr
    # cv.line(map, xy2cv(x1, y1, k), xy2cv(x2, y2, k), random_color, 8)
    cv.arrowedLine(map_img, xy2cv(x1, y1, k), xy2cv(x2, y2, k), random_color, 8, tipLength=0.1)

for node in g.nodes(data=True):
    x, y = node[1]['x'], MAP_H_M-node[1]['y']
    name = node[0]
    cv.circle(map_img, xy2cv(x,y,k), 15, (0, 0, 0), -1)
    cv.circle(map_img, xy2cv(x,y,k), 10, (0, 0, 255), -1)
    #get list of previous and next nodes
    next_nodes = list(g.successors(name))
    assert 0 < len(next_nodes) <= 3, f'node {name} has {len(next_nodes)} next nodes'
    xn, yn = g.nodes[next_nodes[-1]]['x'], MAP_H_M-g.nodes[next_nodes[-1]]['y']
    α = np.arctan2(yn-y, xn-x) + np.pi/2
    d = 0.15 # distance from the node
    xt, yt = x-0.12+d*np.cos(α), y+d*np.sin(α)+0.05
    #add text
    cv.putText(map_img, name, xy2cv(xt,yt,k), cv.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 0), 16)
    cv.putText(map_img, name, xy2cv(xt,yt,k), cv.FONT_HERSHEY_SIMPLEX, 3, (220, 220, 220), 4)

create_window('map')
cv.imshow('map', map_img)
key = cv.waitKey(0)
# check if its enter
if key == 13:
    cv.imwrite(join(this_file, 'stuff', 'final_map.png'), cv.resize(map_img, (map_img.shape[1]//3, map_img.shape[0]//3)))
cv.destroyAllWindows()

In [ ]:
# Create a VerySmall version of the map
import os, cv2 as cv, numpy as np
from os.path import join, exists
this_file = os.path.abspath('.')
map_dir = join(this_file, 'Simulator/src/models_pkg/track/materials/textures/')
map21big = join(map_dir, '2021_Big.png')
map21medium = join(map_dir, '2021_Medium.png')
map21small = join(map_dir, '2021_Small.png')
map21verysmall = join(map_dir, '2021_VerySmall.png')
map24big = join(map_dir, '2024_Big.png')
map24medium = join(map_dir, '2024_Medium.png')
map24small = join(map_dir, '2024_Small.png')

map24verysmall = join(map_dir, '2024_VerySmall.png') # to be created

assert exists(map21big)
assert exists(map21medium)
assert exists(map21small)
assert exists(map21verysmall)
assert exists(map24big)
assert exists(map24medium)
assert exists(map24small)

m = cv.imread(map21big)
print(f'21big: {m.shape}')
r21big = m.shape[0] / m.shape[1] 
m = cv.imread(map21medium)
print(f'21medium: {m.shape}')
r21medium = m.shape[0] / m.shape[1]
m = cv.imread(map21small)
print(f'21small: {m.shape}')
r21small = m.shape[0] / m.shape[1] 
m = cv.imread(map21verysmall)
print(f'21verysmall: {m.shape}')
r21verysmall = m.shape[0] / m.shape[1] 
m = cv.imread(map24big)
print(f'24big: {m.shape}')
r24big = m.shape[0] / m.shape[1] 
m = cv.imread(map24medium)
print(f'24medium: {m.shape}')
r24medium = m.shape[0] / m.shape[1] 
m = cv.imread(map24small)
print(f'24small: {m.shape}')
r24small = m.shape[0] / m.shape[1] 

print(f'ratio 21: {r21big:.3f}, {r21medium:.3f}, {r21small:.3f}, {r21verysmall:.3f}')
print(f'ratio 24: {r24big:.3f}, {r24medium:.3f}, {r24small:.3f}')

#create m24verysmall from m24big, with same height of m21verysmall
h21vs, w21vs = cv.imread(map21verysmall).shape[:2]
m = cv.imread(map24big)
h24b, w24b = m.shape[:2]

h24vs = h21vs
w24vs = int(h24vs * w24b / h24b)

m24vs = cv.resize(m, (w24vs, h24vs))

cv.imwrite(map24verysmall, m24vs)

print(f'h21vs: {h21vs}, w21vs: {w21vs}')
print(f'h24b: {h24b}, w24b: {w24b}')
print(f'h24vs: {h24vs}, w24vs: {w24vs}')

ratio24 = h24b / w24b
print(f'ratio24: {ratio24:.5f}')

#dimensions in meters
hm = 13.6
wm = hm / ratio24
print(f'hm: {hm}, wm: {wm}')



In [ ]:
# compare the maps
import cv2 as cv, numpy as np

m24b_path = 'Simulator/src/models_pkg/track/materials/textures/2024_Big.png'
m24b = cv.imread(m24b_path)
m24m_path = 'Simulator/src/models_pkg/track/materials/textures/2024_Medium_old.png'
m24m_path = 'Simulator/src/models_pkg/track/materials/textures/2024_Medium.png'
m24m = cv.imread(m24m_path)
m24s = cv.imread('Simulator/src/models_pkg/track/materials/textures/2024_Small.png')
m24vs = cv.imread('Simulator/src/models_pkg/track/materials/textures/2024_VerySmall.png')

W, H = 900, 600

maps = [m24b, m24s, m24vs, m24m]
# maps = [m24s, m24vs, m24m]
colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0), (255, 0, 255)]
# colors = [(0, 255, 0), (255, 0, 0), (255, 0, 255)]
#resize all maps to the same size
for i, m in enumerate(maps):
    maps[i] = cv.resize(m, (2*W, 2*H))
    #convert to grayscale
    maps[i] = cv.cvtColor(maps[i], cv.COLOR_BGR2GRAY)
    #convert to binary
    _, maps[i] = cv.threshold(maps[i], 127, 255, cv.THRESH_BINARY)
    #convert to 3 channels
    maps[i] = cv.cvtColor(maps[i], cv.COLOR_GRAY2BGR)
    #change color
    maps[i] = (maps[i].astype(np.float32)/255 * colors[i]).astype(np.uint8)

#add all maps
mapsum = np.zeros((2*H, 2*W, 3), np.uint8)
for m in maps:
    mapsum = cv.add(mapsum, m)

for i, m in enumerate(maps):
    cv.namedWindow(f'map {i}', cv.WINDOW_NORMAL)
    cv.resizeWindow(f'map {i}', W, H)
    cv.imshow(f'map {i}', m)

cv.namedWindow('mapsum', cv.WINDOW_NORMAL)
cv.resizeWindow('mapsum', W, H)
cv.imshow('mapsum', mapsum)

while not cv.waitKey(0) == 27:
    pass

cv.destroyAllWindows()

# create a new 2024_Medium.png from 2024_Big.png
new_m24m_path = 'Simulator/src/models_pkg/track/materials/textures/2024_Medium.png'
m24m_path = 'Simulator/src/models_pkg/track/materials/textures/2024_Medium_old.png'
b = cv.imread(m24b_path)
m = cv.imread(m24m_path)
mh = m.shape[0]
bh, bw = b.shape[:2]
br = bh / bw
new_mw = int(mh / br)
cv.imwrite(new_m24m_path, cv.resize(b, (new_mw, mh)))



In [ ]:
# make maps binary
import os, cv2 as cv, numpy as np
from os.path import join, exists
this_file = os.path.abspath('.')
map_dir = join(this_file, 'Simulator/src/models_pkg/track/materials/textures/')
map24big = join(map_dir, '2024_Big.png')
map24medium = join(map_dir, '2024_Medium.png')
map24small = join(map_dir, '2024_Small.png')
map24verysmall = join(map_dir, '2024_VerySmall.png')

maps = [map24big, map24medium, map24small, map24verysmall]
new_names = ['2024_Big_bin.png', '2024_Medium_bin.png', '2024_Small_bin.png', '2024_VerySmall_bin.png']
assert all([exists(m) for m in maps])

for m, n in zip(maps, new_names):
    print(f'processing {m}...')
    map = cv.imread(m)
    #convert to grayscale
    map = cv.cvtColor(map, cv.COLOR_BGR2GRAY)
    #convert to binary
    _, map = cv.threshold(map, 127, 255, cv.THRESH_BINARY)
    #convert to 3 channels
    map = cv.cvtColor(map, cv.COLOR_GRAY2BGR)
    #save
    cv.imwrite(join(map_dir, n), map)


In [ ]:
# center nodes in the lanes -> create fat map
import os, sys, cv2 as cv, numpy as np
from os.path import join, exists
from tqdm import tqdm

this_file = os.path.abspath('.')
map_path = join(this_file, 'Simulator/src/models_pkg/track/materials/textures/2024_Small_fullclean.png')
assert exists(map_path)

cv.namedWindow('map', cv.WINDOW_NORMAL)
cv.resizeWindow('map', 2560//2-1, 1439)

map = cv.flip(cv.imread(map_path), 0)

#convert to grayscale
map = cv.cvtColor(map, cv.COLOR_BGR2GRAY)
#convert to binary
_, map = cv.threshold(map, 127, 255, cv.THRESH_BINARY)

# kernel = np.ones((9,9),np.uint8)
erode_kernel = np.ones((5,5),np.uint8)
map = cv.erode(map, erode_kernel, iterations=1)

kernel = np.array([[0,0,0,1,0,0,0],
                   [0,1,1,1,1,1,0],
                   [0,1,1,1,1,1,0],
                   [1,1,1,1,1,1,1],
                   [0,1,1,1,1,1,0],
                   [0,1,1,1,1,1,0],
                   [0,0,0,1,0,0,0]], dtype=np.uint8)

# map = cv.erode(map, erode_kernel, iterations=1)
erode_kernel = kernel

N = 20
for i in tqdm(range(N)):
    map = cv.dilate(map, kernel, iterations=10)
    if i != N-1:
        map = cv.blur(map, (13,13))
        _, map = cv.threshold(map, 127, 255, cv.THRESH_BINARY)
        map = cv.erode(map, erode_kernel, iterations=9)
    #show(map)
    cv.imshow('map', cv.flip(map, 0))
    cv.waitKey(1)

#convert to 3 channels
map = cv.cvtColor(map, cv.COLOR_GRAY2BGR)

# cv.imshow('map', cv.flip(map, 0))
# cv.waitKey(0)
cv.destroyAllWindows()

#save fat map
cv.imwrite(join(this_file, 'stuff/2024_Small_fat.png'), cv.flip(map, 0))

###############################################################################################
#center nodes in the lane -> create new graph
import os, sys, cv2 as cv, numpy as np, networkx, matplotlib.pyplot as plt
from os.path import join, exists
from tqdm import tqdm
from stuff import *

this_file = os.path.abspath('.')
g = networkx.read_graphml(join(this_file, 'stuff/Competition_track_graph.graphml'))

# Y_OFF, X_OFF = 0.0, 0.0
Y_OFF, X_OFF = -0.12, 0.0

#add offset to all the nodes
for n in g.nodes(data=True):
    n[1]['x'] += X_OFF
    n[1]['y'] += Y_OFF

#convert y to right hand frame
for n in g.nodes(data=True):
    n[1]['y'] = MAP_H_M - n[1]['y']

# #perrturb x and y of all nodes
# d = 0.002
# for n in g.nodes(data=True):
#     x, y = n[1]['x'], n[1]['y']
#     n[1]['x'], n[1]['y'] = n[1]['x'] + np.random.uniform(-d,d), n[1]['y'] + np.random.uniform(-d,d)

new_g = g.copy() #create a copy of the graph

#load fat map
map_path = join(this_file, 'stuff/2024_Small_fat.png')
k = K_SMALL
assert exists(map_path)

map = cv.flip(cv.imread(map_path), 0)
#convert to binary
map = cv.cvtColor(map, cv.COLOR_BGR2GRAY)
_, map = cv.threshold(map, 127, 255, cv.THRESH_BINARY)
#convert to 3 channels
map_show = cv.cvtColor(map, cv.COLOR_GRAY2BGR)

# show the map
cv.namedWindow('map', cv.WINDOW_NORMAL)
cv.resizeWindow('map', 2560//2-1, 800)
cv.namedWindow('square', cv.WINDOW_NORMAL)
cv.resizeWindow('square', 250, 250)

for n in g.nodes(data=True):
    wait = 1 # ms
    #get the node position
    x, y = n[1]['x'], n[1]['y']
    #see if node is in the middle of 2 nodes
    prev_nodes = list(g.predecessors(n[0]))
    next_nodes = list(g.successors(n[0]))
    is_correct = len(next_nodes) == 1
    color = (255, 0, 0) if is_correct else (0, 0, 255)
    cv.circle(map_show, xy2cv(x,y, k), 60, color, 20)

    if is_correct:
        #get the next node position
        nx, ny = g.nodes[next_nodes[0]]['x'], g.nodes[next_nodes[0]]['y']
        #get the angle between the 2 nodes
        angle = np.arctan2(ny-y, nx-x)
        #get a square around the node
        l = 0.3 #m
        x1, y1, x2, y2 = m2pix(x-l/2, k), m2pix(y-l/2, k), m2pix(x+l/2, k), m2pix(y+l/2, k)
        square = map[y1:y2, x1:x2]
        #rotate the square using a cv2 transformation matrix
        M = cv.getRotationMatrix2D((square.shape[1]//2, square.shape[0]//2), angle*180/np.pi, 1)
        square = cv.warpAffine(square, M, (square.shape[1], square.shape[0]), borderValue=(255,255,255))

        # #draw the square
        # cv.rectangle(map_show, (x1, y1), (x2, y2), (0, 255, 0), 10)

        #check if the square is white
        is_correct = np.mean(square) != 255
        
        #select the central column
        col = square[:, square.shape[1]//2]
        #threshold the column
        col = np.where(col > 127, 255, 0).astype(np.uint8)

        #correct only if both the first and last pixel are white
        is_correct = is_correct and col[0] == 255 and col[-1] == 255

        if is_correct:
            #get the idx of the first black pixel
            first, last = 0, 0
            for i, p in enumerate(col):
                if p == 0 and first == 0: first = i
                if p == 255 and first != 0: last = i; break
            c = (first + last) // 2
            #distance from center to c
            d = pix2m(c - square.shape[1]//2, k)
            #get delta rotated
            angle = angle + np.pi/2
            dx, dy = d*np.cos(angle), d*np.sin(angle)
            #correct the node position
            new_g.nodes[n[0]]['x'] += dx
            new_g.nodes[n[0]]['y'] += dy
            #draw the corrected node
            nx, ny = new_g.nodes[n[0]]['x'], new_g.nodes[n[0]]['y']
            cv.circle(map_show, xy2cv(nx,ny, k), 20, (255, 0, 255), -1)
        
        # cv.imshow('square', square)
        # cv.imshow('map', cv.flip(map_show, 0))
        # # wait = wait if is_correct else 1
        # if cv.waitKey(wait) == 27:
        #     break

    if not is_correct:        
        #draw the square
        cv.rectangle(map_show, (x1, y1), (x2, y2), (255, 255, 0), -1)
        # cv.imshow('map', cv.flip(map_show, 0))
        # cv.imshow('square', square)
        # cv.waitKey(0)
        # p = 10
        # cv.rectangle(map_show, (x1+p, y1+p), (x2-p, y2-p), (0, 0, 0), -1)

#draw all new nodes on the map
for n in new_g.nodes(data=True):
    x, y = n[1]['x'], n[1]['y'] 
    cv.circle(map_show, xy2cv(x,y, k), 50, (0, 0, 255), -1)

#draw all the edges on the map
for e in new_g.edges(data=True):
    x1, y1 = new_g.nodes[e[0]]['x'], new_g.nodes[e[0]]['y']
    x2, y2 = new_g.nodes[e[1]]['x'], new_g.nodes[e[1]]['y']
    cv.line(map_show, xy2cv(x1, y1, k), xy2cv(x2, y2, k), (0, 255, 0), 20)

cv.imshow('map', cv.flip(map_show, 0))
cv.waitKey(0)
cv.destroyAllWindows()

#save the new graph
networkx.write_graphml(new_g, join(this_file, 'stuff/Competition_track_graph_corrected.graphml'))


In [ ]:
# Fix map graph
import os, sys, cv2 as cv, numpy as np, networkx, matplotlib.pyplot as plt
from os.path import join, exists
from stuff import *
STOPLINE_LENGTH = 0.39 #m
this_file = os.path.abspath('.')
# map_name, k = '2024_Big', K_BIG
# map_name, k = '2024_Medium', K_MEDIUM
map_name, k = '2024_Small', K_SMALL
# map_name, k = '2024_VerySmall', K_VERYSMALL
map_path = join(this_file, f'Simulator/src/models_pkg/track/materials/textures/{map_name}.png')
#graphs 
# net_path = join(this_file, 'stuff/Competition_track_graph.graphml')
# net_path = join(this_file, 'stuff/Competition_track_graph_corrected.graphml')
# net_path = join(this_file, 'stuff/final_graph.graphml')
# net_path = join(this_file, 'stuff/Competition_track_graph_corrected_old.graphml')
# net_path = join(this_file, 'stuff/track_graph.graphml')
# new_net_path = join(this_file, 'stuff/track_graph.graphml')
net_path = join(this_file, 'stuff/final_graph.graphml')
new_net_path = join(this_file, 'stuff/final_graph.graphml')

# load map
assert exists(map_path)
map = cv.flip(cv.imread(map_path), 0)
_, clean_map = cv.threshold(cv.cvtColor(map.copy(), cv.COLOR_BGR2GRAY), 127, 255, cv.THRESH_BINARY)

# load graph
G = networkx.read_graphml(net_path)

def draw_nodes_edges(G, map, k):
    #draw all the edges on the map
    for e in G.edges(data=True):
        x1, y1 = G.nodes[e[0]]['x'], G.nodes[e[0]]['y']
        x2, y2 = G.nodes[e[1]]['x'], G.nodes[e[1]]['y']
        random_color = tuple(np.random.randint(100, 255, 3).tolist())  
        # cv.line(map, xy2cv(x1, y1, k), xy2cv(x2, y2, k), (255, 0, 255), 8)
        cv.arrowedLine(map, xy2cv(x1, y1, k), xy2cv(x2, y2, k), random_color, 8, tipLength=0.1)
    #draw all the nodes on the map
    for n in G.nodes(data=True):
        x, y = n[1]['x'], n[1]['y'] 
        cv.circle(map, xy2cv(x,y, k), 18, (0, 0, 0), -1)
        cv.circle(map, xy2cv(x,y, k), 15, (0, 0, 255), -1)
        # cv.putText(map, n[0], xy2cv(x,y, k), cv.FONT_HERSHEY_SIMPLEX, 1, (150, 150, 150), 2)
    return map

map = draw_nodes_edges(G, map, k)

nodes_xy = np.array([[n[1]['x'], n[1]['y']] for n in G.nodes(data=True)])
nodes_names = np.array([n[0] for n in G.nodes(data=True)])

#find closest node to a point
def closest_node(x, y):
    dist_2 = np.sum((nodes_xy - np.array([x, y]))**2, axis=1)
    argmin = np.argmin(dist_2)
    return nodes_names[argmin], nodes_xy[argmin][0], nodes_xy[argmin][1], argmin

create_window('map')

def get_delta_xy(square, angle):
    #rotate the square using a cv2 transformation matrix
    M = cv.getRotationMatrix2D((square.shape[1]//2, square.shape[0]//2), angle*180/np.pi, 1)
    square = cv.warpAffine(square, M, (square.shape[1], square.shape[0]), borderValue=(255,255,255))
    square = np.where(square > 127, 255, 0).astype(np.uint8) #binarize
    #select the central column
    col = square[:, square.shape[1]//2]
    for i, p in enumerate(col):
        if p == 0: c=i; break
    dc = pix2m(c - square.shape[1]//2, k) - STOPLINE_LENGTH/2
    #select the central row
    row = square[square.shape[0]//2,:]
    rowl = (row[:row.shape[0]//2][::-1]/255.0).astype(np.int32)
    rowr = (row[row.shape[0]//2:]/255.0).astype(np.int32)
    dr = pix2m((np.sum(rowr)-np.sum(rowl))//2, k)
    #rotate the delta
    dxr, dyr = dr*np.cos(angle), dr*np.sin(angle)
    dxc, dyc = dc*np.cos(angle + np.pi/2), dc*np.sin(angle + np.pi/2)
    dx, dy = dxc+dxr, dyc+dyr
    return dx, dy

def improve_xy(x, y):
    '''In -> pix, Out -> m'''
    #check if the pixel is white
    if np.mean(clean_map[y, x]) != 255:
        print('not white')
        return pix2m(x, k), pix2m(y, k)
    l = 0.42 #m
    l = m2pix(l, k)
    square = clean_map[y-l//2:y+l//2, x-l//2:x+l//2]
    #find which of the 4 sides is white in the center 
    pr, pl, pt, pb = square[l//2, 0], square[l//2, -1], square[0, l//2], square[-1, l//2]
    #check that only one of the 4 sides is white
    if pt+pb+pl+pr != 255: print('not only one side')
    if pt == 255: angle = 0
    if pb == 255: angle = np.pi
    if pl == 255: angle = np.pi/2
    if pr == 255: angle = -np.pi/2
    dx, dy = get_delta_xy(square, angle)
    x, y = pix2m(x, k) + dx, pix2m(y, k) + dy
    return x,y
    
#create a mouse callback function
def mouse_cb(event, cvx, cvy, flags, param):
    if event == cv.EVENT_LBUTTONDBLCLK:
        x,y = cvx, map.shape[0]-cvy
        xm, ym = pix2m(x, k), pix2m(y, k)
        cv.circle(map, (x, y), 15, (255, 0, 0), 5)
        cv.circle(map, xy2cv(xm, ym, k), 5, (255, 255, 0), -1)
        cv.imshow('map', cv.flip(map, 0))
        x,y = improve_xy(x,y) #in->pix, out->m
        #draw an x on the point
        l = .35 #m
        d = l/2
        #pick a random color
        c = (0,250,0)
        cv.line(map, xy2cv(x-d, y, k), xy2cv(x+d, y, k), c, 2)
        cv.line(map, xy2cv(x, y+d, k), xy2cv(x, y-d, k), c, 2)
        cv.circle(map, xy2cv(x, y, k), m2pix(.39/2,k), c, 5)
        #get closest node
        cn, cx, cy, idx = closest_node(x, y)
        #draw a circle around the closest node
        cv.circle(map, xy2cv(cx, cy, k), 20, c, 5)
        #replace the closest node with the new point
        G.nodes[cn]['x'], G.nodes[cn]['y'] = x, y

#bind the callback function to window
cv.setMouseCallback('map', mouse_cb)

while True:
    cv.imshow('map', cv.flip(map, 0))
    # cv.imshow('map', map)
    #exit if esc or enter are pressed
    key = cv.waitKey(10)
    if key == 27:
        break
    elif key == 13:
        #save the new graph
        networkx.write_graphml(G, new_net_path)
        #redraw the nodes and edges
        map = draw_nodes_edges(G, cv.flip(cv.imread(map_path), 0), k)
        cv.imshow('map', cv.flip(map, 0))
        cv.waitKey(0)
        break

cv.destroyAllWindows()

# #plot the nodes on a plot
# nodes_xy = np.array([[n[1]['x'], n[1]['y']] for n in G.nodes(data=True)])
# nodes_names = np.array([n[0] for n in G.nodes(data=True)])
# plt.scatter(nodes_xy[:, 0], nodes_xy[:, 1])
# plt.show()



In [ ]:
# reposition nodes specular to stoplines, to be done after creating the stop_lines.txt
import os, sys, cv2 as cv, numpy as np, networkx, matplotlib.pyplot as plt
from os.path import join, exists, dirname
from stuff import *
from path_planning import PathPlanning

g = load_graph()
map, _ = load_map()

pp = PathPlanning()

#load stoplines
stoplines = load_nodes(STOPLINES_PATH)

for n in stoplines:
    #get previous and next nodes
    prev_nodes = list(g.predecessors(n))
    x, y = pp.get_xy(n) #get stopline position
    px, py = pp.get_xy(prev_nodes[0]) #get prev node position

    angle = np.arctan2(y-py, x-px) # direction of the stopline
    angle = np.round(angle/np.pi*2)*np.pi/2 #round to the closest multiple of pi/2

    #get new position for specular node
    d = 0.37 #m
    nx, ny = x+d*np.cos(angle+np.pi/2), y+d*np.sin(angle+np.pi/2)

    #draw a circle around the new position
    cv.circle(map, xy2cv(nx, ny), 20, (0, 0, 255), 5)
    cv.circle(map, xy2cv(x, y), 20, (255, 0, 0), 5)

    #get the closest node to the new position
    cn, dist = pp.get_closest_node(nx, ny)
    print(f'sl: {n}, cn: {cn}, dist: {dist:.3f}m')

    dist_thrsh = 0.2 #m

    if dist < dist_thrsh:
        #draw a circle around the closest node
        xcn, ycn = pp.get_xy(cn)
        cv.circle(map, xy2cv(xcn, ycn), 20, (0, 255, 0), 5)
        g.nodes[cn]['x'], g.nodes[cn]['y'] = nx, ny #set the new position


create_window('map')
cv.imshow('map', cv.flip(map, 0))
key = cv.waitKey(0)
if key == 13: #if it's enter
    print('saving graph...')
    networkx.write_graphml(g, GRAPH_PATH) #save the graph
cv.destroyAllWindows()

## Path planning

### Nodes classification

In [ ]:
# find + save + label nodes: like intersections in/out/mid, roundabout, stoplines, save files stuff/
from stuff import *
g = load_graph()
map, k = load_map(VERY_SMALL)

int_mid = ['409'] #409 is not a proper intersection by definition, but should work better this way

for n in g.nodes(data=True):
    x, y = n[1]['x'], n[1]['y']
    xc,yc = 6.031, 7.692 #threshold to filter the roundabout
    name = n[0]
    #get the successors and predecessors
    succ = list(g.successors(name))
    pred = list(g.predecessors(name))
    # if len(succ) >= 2 and len(pred) == 1:
    if len(succ) >= 2 and len(pred) == 1 and (x < xc or y < yc):
        #intersection in
        #add noise to x, y
        σ = 0.1
        x, y = x + np.random.uniform(-σ, σ), y + np.random.uniform(-σ, σ)
        int_mid.append(name)

#draw int_mid
for n in int_mid:
    x, y = g.nodes[n]['x'], g.nodes[n]['y']
    cv.circle(map, xy2cv(x, y, k), 20, (0, 255, 0), 5)

int_in, int_out = [], []
for n in g.nodes(data=True):
    x, y = n[1]['x'], n[1]['y']
    name = n[0]
    #get the successors and predecessors
    succ = list(g.successors(name))
    pred = list(g.predecessors(name))
    for s in succ:
        if s in int_mid:
            #intersection out
            int_in.append(name)
            cv.circle(map, xy2cv(x, y, k), 20, (0, 0, 255), 5)
            break
    for p in pred:
        if p in int_mid:
            #intersection in
            int_out.append(name)
            cv.circle(map, xy2cv(x, y, k), 20, (255, 0, 0), 5)
            break


print(f'int mid: {[int(i) for i in int_mid]}')
print(f'int in: {[int(i) for i in int_in]}')
print(f'int out: {[int(i) for i in int_out]}')

# roundabouts nodes
ra_mid = [str(i) for i in range(600,616)]
ra_in, ra_out = [], []
for n in g.nodes(data=True):
    x, y = n[1]['x'], n[1]['y']
    name = n[0]
    if name in ra_mid: continue
    #get the successors and predecessors
    succ = list(g.successors(name))
    pred = list(g.predecessors(name))
    for s in succ:
        if s in ra_mid:
            #intersection out
            ra_in.append(name)
            cv.circle(map, xy2cv(x, y, k), 20, (0, 0, 155), 5)
            break
    for p in pred:
        if p in ra_mid:
            #intersection in
            ra_out.append(name)
            cv.circle(map, xy2cv(x, y, k), 20, (155, 0, 0), 5)
            break
    
#draw ra_mid
for n in ra_mid:
    x, y = g.nodes[n]['x'], g.nodes[n]['y']
    cv.circle(map, xy2cv(x, y, k), 20, (0, 155, 0), 5)

print(f'ra mid: {[int(i) for i in ra_mid]}')
print(f'ra in: {[int(i) for i in ra_in]}')
print(f'ra out: {[int(i) for i in ra_out]}')

#check if its a stopline
clean_map, k2 = load_map(SMALL)
stoplines = []
def is_stopline(x,y):
    l = 0.35 #m
    l = m2pix(l, k2)
    xpix, ypix = m2pix(x, k2), m2pix(y, k2)
    row = clean_map[ypix, xpix-l//2:xpix+l//2]
    col = clean_map[ypix-l//2:ypix+l//2, xpix]
    row = np.where(row > 127, 255, 0).astype(np.uint8)
    col = np.where(col > 127, 255, 0).astype(np.uint8)
    return np.mean(row) == 255 or np.mean(col) == 255

for n in g.nodes(data=True):
    x, y = n[1]['x'], n[1]['y']
    name = n[0]
    if is_stopline(x,y):
        cv.circle(map, xy2cv(x, y, k), 40, (255, 0, 255), 5)
        stoplines.append(name)

#highway
hwn = [str(i) for i in [*range(193,199), *range(216,237), *range(239,259), *range(261,282), *range(282,302)]] #highway nodes
for n in hwn:
    x, y = g.nodes[n]['x'], g.nodes[n]['y']
    cv.circle(map, xy2cv(x, y, k), 30, (255, 255, 0), 5)
    
print(f'stoplines: {[int(i) for i in stoplines]}')

#show(map)
create_window('map')
cv.imshow('map', cv.flip(map, 0))
cv.waitKey(0)
cv.destroyAllWindows()

# #save all the nodes in different txt files using numpy savetxt
np.savetxt(INT_MID_PATH, int_mid, fmt='%s')
np.savetxt(INT_IN_PATH, int_in, fmt='%s')
np.savetxt(INT_OUT_PATH, int_out, fmt='%s')
np.savetxt(RA_MID_PATH, ra_mid, fmt='%s')
np.savetxt(RA_IN_PATH, ra_in, fmt='%s')
np.savetxt(RA_OUT_PATH, ra_out, fmt='%s')
np.savetxt(STOPLINES_PATH, stoplines, fmt='%s')
np.savetxt(HW_PATH, hwn, fmt='%s')

#test load
print(load_nodes(INT_MID_PATH))
print(load_nodes(INT_IN_PATH))
print(load_nodes(INT_OUT_PATH))
print(load_nodes(RA_MID_PATH))
print(load_nodes(RA_IN_PATH))
print(load_nodes(RA_OUT_PATH))
print(load_nodes(STOPLINES_PATH))
print(load_nodes(HW_PATH))

#redraw everything on a new map to check
map, k = load_map(VERY_SMALL)
cv.namedWindow('map', cv.WINDOW_NORMAL)
cv.resizeWindow('map', 1920//2-1, 800)
for n in load_nodes(INT_MID_PATH): cv.circle(map, xy2cv(g.nodes[n]['x'], g.nodes[n]['y'], k), 20, (0, 255, 0), 5)
for n in load_nodes(INT_IN_PATH): cv.circle(map, xy2cv(g.nodes[n]['x'], g.nodes[n]['y'], k), 20, (255, 0, 0), 5)
for n in load_nodes(INT_OUT_PATH): cv.circle(map, xy2cv(g.nodes[n]['x'], g.nodes[n]['y'], k), 20, (0, 0, 255), 5)
for n in load_nodes(RA_MID_PATH): cv.circle(map, xy2cv(g.nodes[n]['x'], g.nodes[n]['y'], k), 20, (0, 155, 0), 5)
for n in load_nodes(RA_IN_PATH): cv.circle(map, xy2cv(g.nodes[n]['x'], g.nodes[n]['y'], k), 20, (155, 0, 0), 5)
for n in load_nodes(RA_OUT_PATH): cv.circle(map, xy2cv(g.nodes[n]['x'], g.nodes[n]['y'], k), 20, (0, 0, 155), 5)
for n in load_nodes(STOPLINES_PATH): cv.circle(map, xy2cv(g.nodes[n]['x'], g.nodes[n]['y'], k), 40, (255, 0, 255), 5)
for n in load_nodes(HW_PATH): cv.circle(map, xy2cv(g.nodes[n]['x'], g.nodes[n]['y'], k), 30, (255, 255, 0), 5)
cv.imshow('map', cv.flip(map, 0))
cv.waitKey(0)
cv.destroyAllWindows()


### Classify events

In [ ]:
# classify events

#classify every EVENT (stop_line and parking) with its required characteristics 
#the cell shows the stop_points one by one and it's possible to use the keyboard to classify them

import numpy as np, cv2 as cv
from stuff import *

# create event points
#event points are stoplines + parking spote (node 455)
event_points_nodes = [*np.loadtxt(STOPLINES_PATH, dtype=str), '455']
g = load_graph()
event_points = np.array([[g.nodes[n]['x'], g.nodes[n]['y']] for n in event_points_nodes])
#save event points as float32
np.savetxt(EVENT_POINTS_PATH, event_points, fmt='%f')

event_points = np.loadtxt(EVENT_POINTS_PATH, dtype=np.float32)

map_img, _ = load_map()
event_type = np.zeros(len(event_points), dtype=np.int32)

d = {
    'intersection_stop':            ('s',0),
    'intersection_traffic_light':   ('t',1),
    'intersection_priority' :       ('i',2),
    'junction':                     ('j',3),
    'roundabout':                   ('r',4),
    'crosswalk':                    ('c',5),
    'parking':                      ('p',6),
    'highway_exit':                 ('e',7),
}

cv.namedWindow('Classify the point', cv.WINDOW_NORMAL)
# cv.resizeWindow('Classify the point',800,800)
last_selection = 'None'
print('Press the key to select the type of the point')
for k,v in d.items():
    print(f'{k} ==> {v[0]}')
print('Press [ESC] to quit\n')
for i in range(len(event_points)):
    x,y = event_points[i]
    cv.circle(map_img, xy2cv(x,y), 50, (0,0,255), 10)
    cv.imshow('Classify the point', cv.flip(map_img, 0))
    key = cv.waitKey(0)
    if key == 27: # esc
        break
    else :
        done = False
        while not done:
            for k,v in d.items():
                if key == ord(v[0]):
                    event_type[i] = v[1]
                    last_selection = k
                    cv.circle(map_img, m2pix(event_points[i]), 50, (0,255,0), 10)
                    done = True
                    break
            if not done:
                key = cv.waitKey(0)
    print(f'{last_selection}')
cv.destroyAllWindows()

print(event_type)

#save event types as int
np.savetxt(EVENT_TYPES_PATH, event_type, fmt='%d')

### Find signs and classify them

In [ ]:
# GENERATE SIGNS POINTS BY CLICKING ON THE MAP
# then save it in the variable sign_points.npy, aleady in R coord
import numpy as np, cv2 as cv
from stuff import *

wind_name = 'Create Sign Points'
event_points = []

def mouse_callback(event,x,y,flags,param):
    if event == cv.EVENT_LBUTTONDBLCLK:
        xm, ym = pix2m(x), MAP_H_M-pix2m(y)
        p = np.array([xm, ym], dtype=np.float32)
        event_points.append(p)
        cv.circle(map_img, xy2cv(xm,ym), 50, (200,0,200), 10)
        cv.circle(map_img, xy2cv(xm,ym), 3, (200,0,200), -1)

map_img,_ = load_map()
cv.namedWindow(wind_name, cv.WINDOW_NORMAL)
cv.imshow(wind_name, map_img)

cv.setMouseCallback(wind_name, mouse_callback)

while True:
    cv.imshow(wind_name, cv.flip(map_img, 0))
    key = cv.waitKey(1) 
    if key == 27: #ESC
        print('Canceling...')
        break
    if key == 13: #ENTER
        print('Saving...')
        to_save = np.array(event_points)
        np.savetxt(SIGN_POINTS_PATH, to_save, fmt='%f')
        break

cv.destroyAllWindows()

points = np.loadtxt(SIGN_POINTS_PATH, dtype=np.float32)
map_img,_ = load_map()
for i in range(points.shape[0]):
    x,y = points[i]
    cv.circle(map_img, xy2cv(x,y), 30, (0,200,0), 10)
    cv.circle(map_img, xy2cv(x,y), 5, (0,220,0), -1)

cv.namedWindow('signs', cv.WINDOW_NORMAL)
cv.imshow('signs', cv.flip(map_img, 0))
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
#classify every SIGN with its required characteristics 

import numpy as np, cv2 as cv
from stuff import *
sign_points = np.loadtxt(SIGN_POINTS_PATH, dtype=np.float32)
map_img,_ = load_map()
sign_types = np.zeros(len(sign_points))

d = {
    'park':                         ('p',0),
    'closedroad':                   ('d',1),
    'hw_exit':                      ('x',2),
    'hw_enter':                     ('n',3),
    'stop':                         ('s',4),
    'roundabout':                   ('r',5),
    'priority' :                    ('i',6),
    'crosswalk':                    ('c',7),
    'oneway':                       ('o',8),
}

cv.namedWindow('Classify the point', cv.WINDOW_NORMAL)
# cv.resizeWindow('Classify the point',800,800)
last_selection = 'None'
print('Press the key to select the type of the SIGN')
for k,v in d.items():
    print(f'{k} ==> {v[0]}')
print('Press [ESC] to quit\n')
for i in range(len(sign_points)):
    p = sign_points[i]
    cv.circle(map_img, m2pix(p), 50, (0,0,255), 10)
    cv.imshow('Classify the point', map_img)
    key = cv.waitKey(0)
    if key == 27: # esc
        break
    else :
        done = False
        while not done:
            for k,v in d.items():
                if key == ord(v[0]):
                    sign_types[i] = v[1]
                    last_selection = k
                    cv.circle(map_img, m2pix(sign_points[i]), 50, (0,255,0), 10)
                    done = True
                    break
            if not done:
                key = cv.waitKey(0)
    print(f'{last_selection}')
cv.destroyAllWindows()

print(sign_types)

np.savetxt(SIGN_TYPES_PATH, sign_types, fmt='%d')

### Test path planning

In [ ]:
# preliminary tests of path planning
from path_planning2 import PathPlanning
from stuff import *
import os, cv2 as cv, numpy as np, networkx
from os.path import join, exists

this_file = os.path.abspath('.')
map, k = load_map(VERY_SMALL)
g = load_graph()

#get all nodes names
nodes_names = [n[0] for n in g.nodes(data=True)]
names_int = [int(n) for n in nodes_names]
#min and max node
min_node, max_node = min(names_int), max(names_int)
print(f'min node: {min_node}, max node: {max_node}')

path_nodes = [472,207,450,322,122,158,172,247,405,494]
# path_nodes = [472,207,450,322]
# path_nodes = [472,207]

pp = PathPlanning()
pp.generate_path_passing_through(path_nodes)
pp.draw_path()

create_window('pp')
cv.imshow('pp', cv.flip(pp.map,0)) 

cv.waitKey(0)
cv.destroyAllWindows()